In [1]:
# Based on the article:
# https://medium.com/@mrconnor/harnessing-deep-reinforcement-learning-for-algorithmic-trading-d80c36ceda6e

# Intall the required modules
!pip install yfinance
!pip install ta
!pip install stable.baselines3
!pip3 install torch torchvision torchaudio
!pip install cufflinks
!pip install -U gym
#!pip install 'shimmy>=0.2.1'
!pip install wandb -qU

# Import the modules
import gym
from gym.utils import seeding
from gym import spaces

#!pip install TA-Lib
import warnings
import yfinance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import ta
import os
#import talib
warnings.filterwarnings("ignore")
# Optional Plotly Method Imports
import plotly
import cufflinks as cf
cf.go_offline()
from stable_baselines3 import A2C, PPO, DDPG
from stable_baselines3.common.policies import ActorCriticPolicy #MlpPolicy for A2C and PPO
from stable_baselines3.td3.policies import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.noise import ActionNoise, OrnsteinUhlenbeckActionNoise

from datetime import date
from datetime import datetime

sns.set_style('whitegrid')
%matplotlib inline

from datetime import date
from datetime import datetime

sns.set_style('whitegrid')
%matplotlib inline

# Parameters 1 - Stock and holdings

In [2]:
# **** Yahoo Finance API - Stock Parameters ****

# 14 stocks holdings (10 + schwab holdings)
#tic = ['GOOGL', 'MSFT', 'AAPL', 'TSM', 'AMZN', 'META', 'NFLX', 'AMD', 'NVDA', 'CRM', 'PLTR', 'GOLD', 'SNOW', 'TSLA']

#Schwab Holdings
tic = ['AAPL','AMZN','GOLD','GOOG','GOOGL','META','MSFT','SMCI','SNOW','TSLA','TSM','CRM']
Current_Tic_Holdings_list = [53,10,270,2,40,10,10,2,20,2,70,10]

In [3]:
# Create start and end date for stock dataset
todays_date = datetime.now().strftime("%Y-%m-%d")

stock_start_date = '2009-01-01'
#stock_end_date = todays_date

# Create variables to hold info
temp_data=[]
stock_data=pd.DataFrame()

# Download stock data from Yahoo Finance
# and put into dataframe
for s in tic:
  temp_data = yfinance.download(s, start= stock_start_date)
  temp_data['Ticker'] = s
  stock_data = pd.concat([stock_data,temp_data])

# Rename Adj Close
stock_data = stock_data.rename(columns={'Adj Close':'AdjClose'})
# Reset the index
stock_data = stock_data.reset_index()
# Sort the data by Date and Ticker and reset the index
stock_data = stock_data.sort_values(by= ['Date', 'Ticker']).reset_index(drop = True)
#fill NA values with 0
stock_data = stock_data.fillna(0)

# Find all unique dates to avoid duplicates
uniq_date = stock_data.Date.unique()
stocks = pd.DataFrame({"Date": uniq_date})

# Create a copy of stock data and add technical indicators
stock_data_tech = pd.DataFrame()

# Make changes first calculate the technical indicators and then match all the stocks with dates and then backfill NaN with last observed valid data
for i in tic:

  # Force all the date ranges to be same for all stocks #DOW
  temp = pd.merge(stocks, stock_data[stock_data.Ticker == i], how='left', on= 'Date')
  # Fill the missing values
  temp = temp.fillna(method='bfill')
  print('Shape of {} before merging : {} | after merging : {}'.format(i, stock_data[stock_data.Ticker == i].shape, temp.shape))

  # Add all the available technical indicators
  stock_tech = ta.add_all_ta_features(df= temp, open= 'Open', high= 'High', low= 'Low', close='AdjClose', volume = 'Volume', fillna=True)
  stock_data_tech = stock_data_tech._append(stock_tech)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Shape of AAPL before merging : (3879, 8) | after merging : (3879, 8)
Shape of AMZN before merging : (3879, 8) | after merging : (3879, 8)
Shape of GOLD before merging : (3879, 8) | after merging : (3879, 8)
Shape of GOOG before merging : (3879, 8) | after merging : (3879, 8)
Shape of GOOGL before merging : (3879, 8) | after merging : (3879, 8)
Shape of META before merging : (3028, 8) | after merging : (3879, 8)
Shape of MSFT before merging : (3879, 8) | after merging : (3879, 8)
Shape of SMCI before merging : (3879, 8) | after merging : (3879, 8)
Shape of SNOW before merging : (933, 8) | after merging : (3879, 8)
Shape of TSLA before merging : (3505, 8) | after merging : (3879, 8)
Shape of TSM before merging : (3879, 8) | after merging : (3879, 8)
Shape of CRM before merging : (3879, 8) | after merging : (3879, 8)


In [4]:
# Print out the stock data with wanted technical indicators
stock_data_tech[['Date', 'Ticker', 'Open', 'High', 'Low', 'AdjClose', 'Volume','momentum_ppo', 'momentum_rsi','trend_adx', 'trend_macd', 'trend_cci' ]].sample(10)

,Date,Ticker,Open,High,Low,AdjClose,Volume,momentum_ppo,momentum_rsi,trend_adx,trend_macd,trend_cci
1801,2016-03-01,GOOGL,36.064999,37.116501,35.931999,37.108501,60112000.0,-0.359278,55.175378,15.314110,-0.130750,52.482264
2713,2019-10-14,GOLD,17.000000,17.299999,17.000000,15.149158,7369500.0,-0.958327,41.773701,12.751451,-0.150736,-121.984200
1843,2016-04-29,AAPL,23.497499,23.680000,23.127501,21.394478,274126000.0,-2.050796,21.571547,34.336060,-0.489120,-224.972150
683,2011-09-19,AAPL,14.178571,14.758214,14.114286,12.429061,823860800.0,1.655160,68.487172,13.032940,0.191822,196.582966
3827,2024-03-19,META,488.170013,496.630005,481.279999,496.239990,10903100.0,2.662904,58.008836,41.065132,12.785635,-0.861833
3113,2021-05-17,MSFT,246.550003,246.589996,243.520004,238.445709,24970200.0,-0.509508,45.480587,25.970911,-1.233417,-85.065386
1493,2014-12-08,META,76.180000,77.250000,75.400002,76.438896,25733900.0,0.121644,54.585985,15.320886,0.091789,106.202482
443,2010-10-06,AMZN,8.030000,8.034000,7.730000,7.770000,121216000.0,4.179512,60.953946,40.384403,0.310939,61.227770
2055,2017-03-03,CRM,82.000000,82.589996,81.650002,82.112740,6217200.0,1.745831,61.474956,45.196477,1.403127,71.419925
2089,2017-04-21,SMCI,24.700001,24.799999,24.450001,24.600000,262300.0,-1.010327,44.810282,18.247033,-0.252473,-69.320211


In [5]:
stock_data_tech.tail()

,Date,Open,High,Low,Close,AdjClose,Volume,Ticker,volume_adi,volume_obv,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
3874,2024-05-24,270.929993,274.959991,268.529999,272.290009,272.290009,8321600.0,CRM,-1.195009e+09,979765900.0,...,-0.305238,-0.438399,0.133160,1.968511,-7.461292,9.429803,281.246903,-2.254370,-2.280169,3105.710523
3875,2024-05-28,273.100006,273.230011,267.160004,269.829987,269.829987,9294100.0,CRM,-1.196127e+09,970471800.0,...,-0.572773,-0.465273,-0.107499,9.384442,-4.092145,13.476587,280.788268,-0.903457,-0.907562,3076.748320
3876,2024-05-29,268.000000,272.869995,267.769989,271.619995,271.619995,13397900.0,CRM,-1.189297e+09,983869700.0,...,-0.725777,-0.517374,-0.208403,19.407071,0.607698,18.799372,280.572864,0.663384,0.661193,3097.822356
3877,2024-05-30,223.399994,225.089996,212.000000,218.009995,218.009995,66860900.0,CRM,-1.194762e+09,917008800.0,...,-2.406149,-0.895129,-1.511020,57.728383,12.031835,45.696548,257.959282,-19.737133,-21.986310,2466.663893
3878,2024-05-31,219.110001,234.619995,216.093002,234.440002,234.440002,35984705.0,CRM,-1.159477e+09,952993505.0,...,-3.244339,-1.364971,-1.879368,58.297647,21.284997,37.012649,254.292464,7.536355,7.265879,2660.096806


In [6]:
# Display what stocks are in tic list
print(tic)

['AAPL', 'AMZN', 'GOLD', 'GOOG', 'GOOGL', 'META', 'MSFT', 'SMCI', 'SNOW', 'TSLA', 'TSM', 'CRM']


# Parameters 2 - Training Parameters

In [7]:
# Environment Parameters   
ticLen = len(tic)               # number of stocks in tic list
Shares_Per_Trade = 9999         # Maximum shares that can be purchased or sold in a trade
Initial_Investment = 20000      # Initial Investment
Action_Space = ticLen           # Number of stocks

# No. of Initial Investment + No. of Stocks Closing Prices + No. of Shares holding per stock + No. of stocks * No. of Technical Indicators

No_Of_Initial_Investment = 1           # No. of Initial Investment
No_Of_Closing_prices = Action_Space    # No. of Stocks Closing Prices
#No_Of_Shares_Per_Stock = Action_Space  # No. of Shares held per stock

No_Of_Shares_Per_Stock = len(Current_Tic_Holdings_list)

No_Of_Stocks = Action_Space            # No. of stocks
No_Of_Technical_Indicators = 5         # No. of Technical Indicators

Observation_Space = No_Of_Initial_Investment + No_Of_Closing_prices + No_Of_Shares_Per_Stock + No_Of_Stocks * No_Of_Technical_Indicators

Normalized_Rewards = 1e-2       # Factor used to normalized the rewards
commission = 0                  # commission per trade

run_optuma = 'no'              # optimize parameters using Optuma? 

In [8]:
# **** Stock Tracing Env 1 ****

class StockTradingEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, df, Shares_Per_Trade = 10, Initial_Investment = 10000, Action_Space = ticLen, Observation_Space = 57, day = 0,
                 Normalized_Rewards = 1e-4, verbosity = 0, mode = 'train', seed = 10, commission = 0  ):

      self.day = day
      self.df = df
      self.max_shares_per_trade = Shares_Per_Trade
      self.initial_investment = Initial_Investment
      self.Action_Space = Action_Space
      self.Observation_Space = Observation_Space
      self.normalized_rewards = Normalized_Rewards
      self.verbosity = verbosity
      self.commission = commission
      self.mode = mode
      self._seed(seed)

      #************************
      self.ticker = df.Ticker.unique()


      #Action Space
      # Action > 0  means buy shares of stock
      # Action 0 means Hold the stock
      # Action < 0 means sell shares of stock
      self.action_space = spaces.Box(low = -1, high= 1,
                                    shape=(self.Action_Space,), dtype= int)
      #Observation Space
      self.observation_space = spaces.Box(low = -np.inf, high= np.inf, shape=(self.Observation_Space,))
      #Selecting the Data for one date
      self.data = self.df.loc[self.day,:]
      #Initial Run
      self.initial = True
      #Verify if tradings days are completed or not
      self.done = False
      #Rewards
      self.reward = 0
      #Asset value after each trading day
      self.asset_memory = [self.initial_investment]
      #Rewards received for each trading day i.e profit or loss
      self.reward_memory = []
      #Saving the date for the trade
      self.date_memory = [self.data.Date.unique()[0]]
      #Initializing state of the environment
      self.state = [self.initial_investment] + self.data.AdjClose.values.tolist() + Current_Tic_Holdings_list + \
        self.data.momentum_ppo.values.tolist() + self.data.momentum_rsi.values.tolist() + \
        self.data.trend_adx.values.tolist() + self.data.trend_macd.values.tolist() + \
        self.data.trend_cci.values.tolist()

      #************************
      self.keep_row_info = {}
      self.keep_rows_list = []

    def render(self, mode='human'):
      return self.state

    # This method is used to reset the values of the state to it's default after every episode
    def reset(self):
      #print(" ")
      #print("reseting self...")

      self.day = 0
      self.reward = 0
      self.data = self.df.loc[self.day,:]
      self.done = False
      self.initial = False
      self.reward_memory = []
      self.date_memory = [str(self.data.Date.unique()[0])]
      self.asset_memory = [self.initial_investment]
      #***Technical Indicators tolist (check the  observation_space num !!!)
      #***Search "self.state ="
      self.state = [self.initial_investment] + self.data.AdjClose.values.tolist() + Current_Tic_Holdings_list + \
        self.data.momentum_ppo.values.tolist() + self.data.momentum_rsi.values.tolist() + \
        self.data.trend_adx.values.tolist() + self.data.trend_macd.values.tolist() + \
        self.data.trend_cci.values.tolist()
    
      #************************
      self.keep_row_info = {}
      self.keep_rows_list = []
      #print(self.state)
      return self.state

    def step(self, actions):
      #************************
      #print(' ')
      #print('************--start--************')



      self.done = self.day >= len(self.df.Date.unique())-1
      #Use this to save the results to csv after we performed trading for all the days
      if self.done:
        #@@@@@
        keep_df = pd.DataFrame(index = [x for x in range(len(self.keep_rows_list))],data = self.keep_rows_list)

        #print('Keep DF')
        #print(keep_df)

        final_portfolio_value = self.state[0] + sum(np.array(self.state[1:self.Action_Space+1])*
                                                     np.array(self.state[self.Action_Space+1:(2*self.Action_Space)+1]))

        total_rewards = final_portfolio_value - self.initial_investment
        profit_pct = (total_rewards*100)/self.initial_investment
        asset_df = pd.DataFrame(self.asset_memory)
        asset_df.columns = ['portfolio']
        asset_df['date'] = self.date_memory
        #if self.verbosity and self.mode != 'train':
        if self.verbosity :
          #print(len(self.reward_memory))
          #if self.mode == 'trade' or self.mode == 'val':

          asset_df['date'] = self.date_memory


          print( 'Initial Portfolio Value : {} | Final Portfolio Value : {} | Total rewards : {} | % of profit : {}'.format(self.initial_investment, final_portfolio_value,total_rewards, profit_pct))
          asset_df.to_csv('{}_{}_results.csv'.format(self.mode, self.commission))



        #************************
        #keep_df = pd.DataFrame(keep_day_list, columns=['date'], index = [x for x in range(len(keep_day_list))])
        #keep_df['portfolio_before_trade'] = keep_portfolio_before_trade_list
        #keep_df['available_cash_after_trading'] = keep_available_cash_after_trading_list
        #keep_df['shares_available_per_stock_list'] = keep_shares_available_per_stock_list
        #keep_df['profit_and_loss_for_day'] = keep_profit_and_loss_for_day

        #print(keep_df)

        #keep_df.to_csv('keep_results.csv')

        return self.state, self.reward, self.done, {}

      else:

        #Calculating the portfolio value before start of trading
        #Available investment amount + sum of value of each stock held (no.of shares per stock * price of the stock on that day)
        portfolio_before_trade = self.state[0] + sum(np.array(self.state[1:self.Action_Space+1])*
                                                     np.array(self.state[self.Action_Space+1:(2*self.Action_Space)+1]))

        #************************
        keep_portfolio_before_trade = portfolio_before_trade
        #keep_portfolio_before_trade_list.append(keep_portfolio_before_trade)
        keep_available_cash_before_trading = self.state[0]

        #Extracting the indicies of sell action
        #if actions is not an array then convert it to array using np.array()
        sell_indices = np.where(actions <  0 )[0]
        #Extracting the indicies of buy action
        buy_indices = np.where(actions >  0 )[0]


        #************************
        #Hold indicies
        hold_indices = np.where(actions ==  0 )[0]



        #print(actions)
        #print(self.ticker)


        ###### Trading starts #######
        # Initially selling the stocks to increase investment value
        for idx in sell_indices:

          #************************
          #print("Stock to sell: " + str(idx))
          sell_ticker_symbol = self.ticker[idx]
          #print('Ticker: ' + str(sell_ticker_symbol))

          stock_to_sell_ticker = 'Stock_to_sell_' + str(sell_ticker_symbol)
          self.keep_row_info[stock_to_sell_ticker] = 'Yes'


          #Sell stock if price is > 0 and shares held > 0
          if self.state[idx+1] > 0 and self.state[idx+self.Action_Space+1] > 0:

            #No of shares to sell
            #No of shares to sell
            #No of shares to sell
            shares_sell = min(self.state[idx+self.Action_Space+1], abs(actions[idx]*self.max_shares_per_trade))
            #shares_sell = int(min((self.state[idx+self.Action_Space+1]//1000)*1000, abs(actions[idx]*self.max_shares_per_trade))//1000)*1000

            num_of_ticker_shares_available_before_selling = 'num_of_' + str(sell_ticker_symbol) + 'shares_available_before_selling'
            self.keep_row_info[num_of_ticker_shares_available_before_selling] = str(self.state[idx+self.Action_Space+1])

            #************************
            #print('No. of shares to sell: ' + str(shares_sell))
            num_of_ticker_to_sell = 'num_of_' + str(sell_ticker_symbol) + '_to_sell:'
            self.keep_row_info[num_of_ticker_to_sell] = str(shares_sell)

            #Updating the available cash after selling the stocks
            self.state[0] += self.state[idx+1]*shares_sell*(1-self.commission)

            #Updating the available stocks after selling
            self.state[idx+self.Action_Space+1] -= shares_sell

            #************************
            #print('No. of shares available after selling: ' + str(self.state[idx+self.Action_Space+1]))
            num_of_ticker_shares_available_after_selling = 'num_of_' + str(sell_ticker_symbol) + 'shares_available_after_selling'
            self.keep_row_info[num_of_ticker_shares_available_after_selling] = str(self.state[idx+self.Action_Space+1])


            #stock_info_df = self.df
            #wanted_df_1 = stock_info_df[stock_info_df['Date'] == date_string]
            #wanted_df_2 = wanted_df_1[wanted_df_1['Ticker'] == sell_ticker_symbol]
            #wanted_df_2 = wanted_df_2[['Date', 'Ticker', 'Open', 'High', 'Low', 'AdjClose', 'Volume','momentum_ppo', 'momentum_rsi','trend_adx', 'trend_macd', 'trend_cci' ]]

#@@@
            #for w_column in wanted_df_2.columns.tolist():
              #print(str(sell_ticker_symbol))
              #print(str(w_column))
              #print(str(wanted_df_2[w_column].values.tolist()[0]))
              #print(str(sell_ticker_symbol) + '_' + str(w_column)+ ' : ' + str(wanted_df_2[w_column].values.tolist()))

            #  self.keep_row_info[str(sell_ticker_symbol) + '_' + str(w_column)] = str(wanted_df_2[w_column].values.tolist()[0])
              #pass



          else:
            # print('No Shares to sell')
            pass
        #print('Buying Stock shares : ')
        for idx in buy_indices:

          #************************
          #print("Stock to buy: " + str(idx))
          buy_ticker_symbol = self.ticker[idx]
          #print('Ticker: ' + str(buy_ticker_symbol))

          stock_to_buy_ticker = 'Stock_to_buy_' + str(buy_ticker_symbol)
          self.keep_row_info[stock_to_buy_ticker] = 'Yes'

          #Buy stocks if price is > 0
          if self.state[idx+1] > 0 and self.state[0] > 0:

            #Max number of shares that can be brought with the available cash (available cash / stock price)
            max_shares_buy = self.state[0]*(1 - self.commission)//self.state[idx+1]
            #max_shares_buy = int(self.state[0]*(1 - self.commission)//self.state[idx+1]//1000)*1000

            #No of shares to buy
            shares_buy = min(max_shares_buy, actions[idx]*self.max_shares_per_trade)
            #shares_buy = ((min(max_shares_buy, actions[idx]*self.max_shares_per_trade))//1000)*1000

            num_of_ticker_shares_available_before_buying = 'num_of_' + str(buy_ticker_symbol) + 'shares_available_before_buying'
            self.keep_row_info[num_of_ticker_shares_available_before_buying] = str(self.state[idx+self.Action_Space+1])

            #************************
            #print('No. of shares to buy: ' + str(shares_buy))
            num_of_ticker_to_buy = 'num_of_' + str(buy_ticker_symbol) + '_to_buy:'
            self.keep_row_info[num_of_ticker_to_buy] = str(shares_buy)

            #Updating the available cash after buying the stocks
            self.state[0] -= self.state[idx+1]*shares_buy*(1 + self.commission)

            #Updating the available stocks after buying
            self.state[idx+self.Action_Space+1] += shares_buy

            #************************
            #print('No. of shares available after buying: ' + str(self.state[idx+self.Action_Space+1]))
            num_of_ticker_shares_available_after_buying = 'num_of_' + str(buy_ticker_symbol) + 'shares_available_after_buying'
            self.keep_row_info[num_of_ticker_shares_available_after_buying] = str(self.state[idx+self.Action_Space+1])

            #stock_info_df = self.df
            #wanted_df_1 = stock_info_df[stock_info_df['Date'] == date_string]
            #wanted_df_2 = wanted_df_1[wanted_df_1['Ticker'] == buy_ticker_symbol]
            #wanted_df_2 = wanted_df_2[['Date', 'Ticker', 'Open', 'High', 'Low', 'AdjClose', 'Volume','momentum_ppo', 'momentum_rsi','trend_adx', 'trend_macd', 'trend_cci' ]]

#@@@
            #for w_column in wanted_df_2.columns.tolist():
              #print(str(sell_ticker_symbol))
              #print(str(w_column))
              #print(str(wanted_df_2[w_column].values.tolist()[0])
              #print(str(sell_ticker_symbol) + '_' + str(w_column)+ ' : ' + str(wanted_df_2[w_column].values.tolist()))

            #  self.keep_row_info[str(buy_ticker_symbol) + '_' + str(w_column)] = str(wanted_df_2[w_column].values.tolist()[0])
              #pass

          else:

            #************************
            for idx in hold_indices:
              #print("Stock to hold: " + str(idx))
              hold_ticker_symbol = self.ticker[idx]
              #print('Ticker: ' + str(hold_ticker_symbol))

              stock_to_hold_ticker = 'Stock_to_hold_' + str(hold_ticker_symbol)
              self.keep_row_info[stock_to_hold_ticker] = 'Yes'

              num_of_ticker_shares_available_for_holding = 'num_of_' + str(hold_ticker_symbol) + 'shares_available_for_holding'
              self.keep_row_info[num_of_ticker_shares_available_for_holding] = str(self.state[idx+self.Action_Space+1])

              #stock_info_df = self.df
              #wanted_df_1 = stock_info_df[stock_info_df['Date'] == date_string]
              #wanted_df_2 = wanted_df_1[wanted_df_1['Ticker'] == hold_ticker_symbol]
              #wanted_df_2 = wanted_df_2[['Date', 'Ticker', 'Open', 'High', 'Low', 'AdjClose', 'Volume','momentum_ppo', 'momentum_rsi','trend_adx', 'trend_macd', 'trend_cci' ]]

#@@@
              #for w_column in wanted_df_2.columns.tolist():
                #print(str(sell_ticker_symbol))
                #print(str(w_column))
                #print(str(wanted_df_2[w_column].values.tolist()[0]))
                #print(str(hold_ticker_symbol) + '_' + str(w_column)+ ' : ' + str(wanted_df_2[w_column].values.tolist()))

              #  self.keep_row_info[str(hold_ticker_symbol) + '_' + str(w_column)] = str(wanted_df_2[w_column].values.tolist()[0])
                #pass

            # print('No Shares purchased')
            pass
        ###### Trading ends #######
        # print('*************** Trading Ends ***************')
        # print('Available cash for after trading : {}'.format(self.state[0]))
        # print('Shares available per stock : '.format(np.array(self.state[self.Action_Space+1:(2*self.Action_Space)+1])))
        # print('Over all portfolie before Trading : {}'.format(portfolio_after_trade))
        # print('Profit or Loss for Day {} : {} is {}'.format(self.day, self.date_memory[-1], self.reward))

        #************************
        keep_available_cash_after_trading = self.state[0]
        keep_shares_available_per_stock = np.array(self.state[self.Action_Space+1:(2*self.Action_Space)+1])
        keep_profit_and_loss_for_day = self.reward
        #keep_available_cash_after_trading_list.append(keep_available_cash_after_trading)
        #keep_shares_available_per_stock_list.append(keep_shares_available_per_stock)
        #keep_profit_and_loss_for_day_list.append(Keep_profit_and_loss_for_day)




        #Setting the values for next trading date
        self.day += 1

        #************************


        self.data = self.df.loc[self.day,:]

        self.state = [self.state[0]] + self.data.AdjClose.values.tolist() + list(self.state[(self.Action_Space+1):(2*self.Action_Space)+1]) + \
                      self.data.momentum_ppo.values.tolist() + self.data.momentum_rsi.values.tolist() + \
                      self.data.trend_adx.values.tolist() + self.data.trend_macd.values.tolist() + \
                      self.data.trend_cci.values.tolist()


        portfolio_after_trade = self.state[0] + sum(np.array(self.state[1:self.Action_Space+1])*
                                                     np.array(self.state[self.Action_Space+1:(2*self.Action_Space)+1]))

        #************************
        keep_portfolio_after_trade = portfolio_after_trade
        #print('Portfolio value after trading:  ' + str(keep_portfolio_after_trade))
        #print('Available Cash before trading:  ' + str(keep_available_cash_before_trading))
        #print('Available Cash after trading:   ' + str(keep_available_cash_after_trading))
        #print('Profit and loss for day:        ' + str(keep_profit_and_loss_for_day))

        self.keep_row_info['portfolio_value_after_trading'] = str(keep_portfolio_after_trade)
        self.keep_row_info['available_cash_before_trading'] = str(keep_available_cash_before_trading)
        self.keep_row_info['available_cash_after_trading'] = str(keep_available_cash_after_trading)
        self.keep_row_info['profit_and_loss_for_day'] = str(keep_profit_and_loss_for_day)

        #Total trade in a day (profit or loss)
        self.reward = portfolio_after_trade - portfolio_before_trade
        #print('Day : {} | Reward : {}'.format(self.day-1, self.reward))
        self.reward_memory.append(self.reward)
        self.asset_memory.append(portfolio_after_trade)
        self.date_memory.append(self.data.Date.unique()[0])
        self.reward = self.reward*self.normalized_rewards     #Normalizing the reward

        self.keep_rows_list.append(self.keep_row_info)
        #print(self.keep_rows_list)

        return self.state, self.reward, self.done, {}

    def _seed(self, seed = 10):
      randomState, seed = seeding.np_random(seed)
      return [seed]

    #added for seed
    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

def render_trading(model, env, data, n_episodes = 1):
  episode_rewards = [0.0]
  obs = env.reset()
  env.render()

  for i in range(n_episodes):
    done = False
    while not done:
        action, _states = model.predict(obs)
        #print(action)
        obs, rewards, done, info = env.step(action)
        env.render()
        # Stats
        if done:
          obs = env.reset()
          #print('Episode {} Rewards {}'.format(i+1, episode_rewards[-1]))
          episode_rewards.append(0.0)
        else:
          #print(rewards)
          episode_rewards[-1] += rewards
    if (i+1)%100 == 0 and i != 0:
      print('Average reward {}'.format(np.average(episode_rewards[:i+1])))

  return episode_rewards

In [9]:
#Find the date 2 days before today's date
from datetime import datetime, timedelta

# Convert string to datetime object
date_object = datetime.strptime(todays_date, '%Y-%m-%d')

# Subtract 2 days
new_date_object = date_object - timedelta(days=2)

# Convert back to string
twoDaysBeforeTodays_date = new_date_object.strftime('%Y-%m-%d')

stock_df = stock_data_tech.copy()
#Training Data - 2009 --> end of 2020
train_data = stock_df[(stock_df.Date >= '2009-01-01') & (stock_df.Date < '2024-04-30')].sort_values(by=['Date', 'Ticker'])

#Validation Data 2021 --> mar 29 2022
validation_data = stock_df[(stock_df.Date >= '2022-01-01') & (stock_df.Date < twoDaysBeforeTodays_date)]
validation_data.sort_values(["Date", "Ticker"], ignore_index=True)
validation_data.index = validation_data.Date.factorize()[0]

#Trading Data - 2 days before current date
trading_data = stock_df[(stock_df.Date >= twoDaysBeforeTodays_date)]
trading_data.sort_values(["Date", "Ticker"], ignore_index=True)
trading_data.index = trading_data.Date.factorize()[0]

In [10]:
twoDaysBeforeTodays_date

'2024-05-29'

In [11]:
trading_data

,Date,Open,High,Low,Close,AdjClose,Volume,Ticker,volume_adi,volume_obv,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,2024-05-29,189.610001,192.250000,189.509995,190.289993,190.289993,53068000.0,AAPL,-1.279613e+13,3.631844e+10,...,2.377611,2.435407,-0.057795,-9.055103,-7.380385,-1.674718,189.398764,0.157897,0.157772,6844.452754
1,2024-05-30,190.759995,192.179993,190.630005,191.289993,191.289993,49947900.0,AAPL,-1.279614e+13,3.636838e+10,...,2.326334,2.413592,-0.087258,-8.530273,-7.610363,-0.919910,189.435877,0.525514,0.524138,6880.946805
2,2024-05-31,191.440002,192.570007,189.910004,192.250000,192.250000,74776921.0,AAPL,-1.279608e+13,3.644316e+10,...,2.299492,2.390772,-0.091280,-4.263600,-6.941010,2.677410,189.520314,0.501859,0.500604,6915.981339
0,2024-05-29,181.699997,184.080002,181.550003,182.020004,182.020004,32009300.0,AMZN,2.219502e+10,1.803329e+10,...,0.026981,0.397175,-0.370194,-8.390572,-4.637696,-3.752877,182.471659,-0.071364,-0.071390,6596.836226
1,2024-05-30,181.309998,181.339996,178.360001,179.320007,179.320007,29249200.0,AMZN,2.218462e+10,1.800404e+10,...,-0.137938,0.290152,-0.428090,-9.240997,-5.558356,-3.682641,181.940689,-1.483352,-1.494463,6497.498587
2,2024-05-31,178.229996,179.210007,173.869995,176.440002,176.440002,58350991.0,AMZN,2.218243e+10,1.794569e+10,...,-0.392337,0.153654,-0.545991,-3.457333,-5.138151,1.680818,180.963708,-1.606070,-1.619107,6391.538140
0,2024-05-29,17.240000,17.379999,16.910000,16.920000,16.920000,12268800.0,GOLD,-4.957435e+11,1.172969e+09,...,0.686866,0.942282,-0.255416,-9.424412,-9.927313,0.502901,17.631396,-2.981654,-3.027009,-39.502739
1,2024-05-30,16.959999,17.209999,16.959999,17.110001,17.110001,12306400.0,GOLD,-4.957410e+11,1.185276e+09,...,0.582508,0.870328,-0.287819,-11.303681,-10.202586,-1.101095,17.622047,1.122935,1.116676,-38.823394
2,2024-05-31,17.129999,17.260000,16.969999,17.090000,17.090000,15459464.0,GOLD,-4.957437e+11,1.169816e+09,...,0.484825,0.793227,-0.308402,-11.289881,-10.420045,-0.869836,17.607786,-0.116893,-0.116962,-38.894905
0,2024-05-29,176.809998,178.229996,176.259995,177.399994,177.399994,15023800.0,GOOG,7.311319e+09,9.374980e+09,...,2.549876,2.777653,-0.227777,-11.097125,-8.886391,-2.210734,174.172812,-0.348281,-0.348889,2116.667809


In [12]:
#if length od trading data is < 22, descend down the dates until 22

trading_data = stock_df[(stock_df.Date >= twoDaysBeforeTodays_date)]
current_date = twoDaysBeforeTodays_date
if len(trading_data) > 22:
    while len(trading_data) > 22:
        ### Count down dates until there len(trading_data) >= 22 ###    
    
        # Convert string to datetime object
        current_date_object = datetime.strptime(current_date, '%Y-%m-%d')
        # Subtract 1 day
        new_new_date_object = current_date_object + timedelta(days=1)
        # Convert back to string
        new_new_date = new_new_date_object.strftime('%Y-%m-%d')
        current_date =  new_new_date

        trading_data = stock_df[(stock_df.Date >= new_new_date)]    
        trading_data.sort_values(["Date", "Ticker"], ignore_index=True)
        trading_data.index = trading_data.Date.factorize()[0]
    
        print(current_date)
        print(len(trading_data))
        print(' ')
        
elif len(trading_data) < 22:
    while len(trading_data) < 22:
        ### Count down dates until there len(trading_data) >= 22 ###    
    
        # Convert string to datetime object
        current_date_object = datetime.strptime(current_date, '%Y-%m-%d')
        # Subtract 1 day
        new_new_date_object = current_date_object - timedelta(days=1)
        # Convert back to string
        new_new_date = new_new_date_object.strftime('%Y-%m-%d')
        current_date =  new_new_date
        
        trading_data = stock_df[(stock_df.Date >= new_new_date)]    
        trading_data.sort_values(["Date", "Ticker"], ignore_index=True)
        trading_data.index = trading_data.Date.factorize()[0]
    
        print(current_date)
        print(len(trading_data))
        print(' ')
else:
    pass

2024-05-30
24
 
2024-05-31
12
 


In [13]:
print(trading_data['Date'].head(1))
print(trading_data['Date'].tail(1))

0   2024-05-31
Name: Date, dtype: datetime64[ns]
0   2024-05-31
Name: Date, dtype: datetime64[ns]


In [14]:
from datetime import datetime, timedelta
new3_date = trading_data['Date'].head(1).iloc[0].strftime('%Y-%m-%d')
twoDaysBeforeTodays_date = new3_date

In [15]:
twoDaysBeforeTodays_date

'2024-05-31'

In [16]:
#Training Environment
train_env = DummyVecEnv([lambda : StockTradingEnv(train_data, Shares_Per_Trade,
                                                  Initial_Investment, Action_Space,
                                                  Observation_Space, verbosity=1, mode='train',
                                                  Normalized_Rewards= Normalized_Rewards,
                                                  commission = commission)])

train_env = VecNormalize(train_env, norm_reward=False)

#Validation Environment
val_env = DummyVecEnv([lambda : StockTradingEnv(validation_data, Shares_Per_Trade,
                                                Initial_Investment, Action_Space,
                                                Observation_Space, verbosity=1, mode='val',
                                                Normalized_Rewards= Normalized_Rewards,
                                                commission = commission)])

val_env = VecNormalize(val_env, norm_reward=False)

#Trading Environment
trade_env = DummyVecEnv([lambda : StockTradingEnv(trading_data, Shares_Per_Trade,
                                                  Initial_Investment, Action_Space,
                                                  Observation_Space, verbosity=1, mode='trade',
                                                  Normalized_Rewards= Normalized_Rewards,
                                                  commission = commission)])

trade_env = VecNormalize(trade_env, norm_reward=False)

In [17]:
# **** Setup A2C Model ****

import tensorflow as tf
import datetime, os
from stable_baselines3 import A2C
from stable_baselines3.common.callbacks import EvalCallback,StopTrainingOnRewardThreshold

a2c_model = A2C('MlpPolicy', env = train_env, seed= 10,
                tensorboard_log= './logs',
                verbose= 1, learning_rate= 0.0005, ent_coef= 0.05, n_steps=5, gamma=0.1,max_grad_norm=0.5)

logdir = os.path.join("/content/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Setup the evaluation callback
save_path=r"/content/best_model"
stop_callback=StopTrainingOnRewardThreshold(reward_threshold=50000,verbose=1)
eval_callback = EvalCallback(eval_env=train_env,callback_on_new_best=stop_callback, best_model_save_path=save_path,
                             log_path='./logs', eval_freq=2000,verbose=1,
                             deterministic=True, render=False)

# Pass the callback to the model's learn method
a2c_model.learn(total_timesteps=50000, log_interval= 1000, callback=eval_callback)

Using cpu device
Logging to ./logs\A2C_250
Initial Portfolio Value : 20000 | Final Portfolio Value : 1378054.6353661441 | Total rewards : 1358054.6353661441 | % of profit : 6790.273176830721
Initial Portfolio Value : 20000 | Final Portfolio Value : 2581834.3919900176 | Total rewards : 2561834.3919900176 | % of profit : 12809.171959950088
Initial Portfolio Value : 20000 | Final Portfolio Value : 2478075.233029923 | Total rewards : 2458075.233029923 | % of profit : 12290.376165149613
Initial Portfolio Value : 20000 | Final Portfolio Value : 2343305.0760615375 | Total rewards : 2323305.0760615375 | % of profit : 11616.525380307688
Initial Portfolio Value : 20000 | Final Portfolio Value : 2894919.8690124396 | Total rewards : 2874919.8690124396 | % of profit : 14374.599345062199
Eval num_timesteps=2000, episode_reward=23283.87 +/- 5157.92
Episode length: 3856.00 +/- 0.00
------------------------------------
| eval/                 |          |
|    mean_ep_length     | 3.86e+03 |
|    mean_

Initial Portfolio Value : 20000 | Final Portfolio Value : 1666602.0527273323 | Total rewards : 1646602.0527273323 | % of profit : 8233.010263636663
Initial Portfolio Value : 20000 | Final Portfolio Value : 1636922.4248760287 | Total rewards : 1616922.4248760287 | % of profit : 8084.612124380143
Eval num_timesteps=12000, episode_reward=16528.28 +/- 179.63
Episode length: 3856.00 +/- 0.00
------------------------------------
| eval/                 |          |
|    mean_ep_length     | 3.86e+03 |
|    mean_reward        | 1.65e+04 |
| time/                 |          |
|    total_timesteps    | 12000    |
| train/                |          |
|    entropy_loss       | -17.1    |
|    explained_variance | 0.0477   |
|    learning_rate      | 0.0005   |
|    n_updates          | 2399     |
|    policy_loss        | -399     |
|    std                | 1        |
|    value_loss         | 1.27e+03 |
------------------------------------
Initial Portfolio Value : 20000 | Final Portfolio Value

Initial Portfolio Value : 20000 | Final Portfolio Value : 1249016.1348215938 | Total rewards : 1229016.1348215938 | % of profit : 6145.080674107969
Initial Portfolio Value : 20000 | Final Portfolio Value : 1249987.323712637 | Total rewards : 1229987.323712637 | % of profit : 6149.936618563185
Initial Portfolio Value : 20000 | Final Portfolio Value : 1251382.6761536445 | Total rewards : 1231382.6761536445 | % of profit : 6156.913380768222
Initial Portfolio Value : 20000 | Final Portfolio Value : 1252166.8139146667 | Total rewards : 1232166.8139146667 | % of profit : 6160.834069573333
Initial Portfolio Value : 20000 | Final Portfolio Value : 1252360.5264100055 | Total rewards : 1232360.5264100055 | % of profit : 6161.802632050028
Eval num_timesteps=24000, episode_reward=12361.14 +/- 12.98
Episode length: 3856.00 +/- 0.00
------------------------------------
| eval/                 |          |
|    mean_ep_length     | 3.86e+03 |
|    mean_reward        | 1.24e+04 |
| time/              

Initial Portfolio Value : 20000 | Final Portfolio Value : 1653673.1533735339 | Total rewards : 1633673.1533735339 | % of profit : 8168.365766867669
Eval num_timesteps=34000, episode_reward=16523.11 +/- 169.56
Episode length: 3856.00 +/- 0.00
------------------------------------
| eval/                 |          |
|    mean_ep_length     | 3.86e+03 |
|    mean_reward        | 1.65e+04 |
| time/                 |          |
|    total_timesteps    | 34000    |
| train/                |          |
|    entropy_loss       | -17.4    |
|    explained_variance | 0.0188   |
|    learning_rate      | 0.0005   |
|    n_updates          | 6799     |
|    policy_loss        | -194     |
|    std                | 1.03     |
|    value_loss         | 5e+03    |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 96       |
|    iterations         | 7000     |
|    time_elapsed       | 362      |
|    total_timeste

------------------------------------
| time/                 |          |
|    fps                | 95       |
|    iterations         | 9000     |
|    time_elapsed       | 470      |
|    total_timesteps    | 45000    |
| train/                |          |
|    entropy_loss       | -17.5    |
|    explained_variance | 0.23     |
|    learning_rate      | 0.0005   |
|    n_updates          | 8999     |
|    policy_loss        | 102      |
|    std                | 1.04     |
|    value_loss         | 221      |
------------------------------------
Initial Portfolio Value : 20000 | Final Portfolio Value : 2846021.08740098 | Total rewards : 2826021.08740098 | % of profit : 14130.1054370049
Initial Portfolio Value : 20000 | Final Portfolio Value : 2848785.706541843 | Total rewards : 2828785.706541843 | % of profit : 14143.928532709215
Initial Portfolio Value : 20000 | Final Portfolio Value : 2849517.480747158 | Total rewards : 2829517.480747158 | % of profit : 14147.58740373579
Initial P

# Optimize Parameters

In [18]:
# Connnect to Weights and Biases
import wandb
wandb.login()

#fa012b2b2fd4bb72581ff4f975f151c2c74bae6a

wandb: Currently logged in as: silvergreen333. Use `wandb login --relogin` to force relogin


True

In [19]:
from wandb.integration.sb3 import WandbCallback
config = {
    "policy_type": "MlpPolicy",
    "total_timesteps": 50000,
    "env_name": train_env,
}

hyperparams = dict(
    n_steps=5, # number of steps to collect data before updating policy
    learning_rate=0.0005,
    gamma=0.1, # discount factor
    max_grad_norm=0.5, # The maximum value for the gradient clipping
    ent_coef=0.05, # Entropy coefficient for the loss calculation
)

"""
hyperparams = dict(
    n_steps=7, # number of steps to collect data before updating policy
    learning_rate=0.001003582768121682,
    gamma=0.0018126757076625249, # discount factor
    max_grad_norm=0.2819622027161751, # The maximum value for the gradient clipping
    ent_coef=0.05, # Entropy coefficient for the loss calculation
)
"""


run = wandb.init(
    project="20240421_US_STOCK_experiment",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    monitor_gym=True,  # auto-upload the videos of agents playing the game
    save_code=True,  # optional
)

a2c_model=A2C("MlpPolicy", train_env, seed=10, verbose=1, **hyperparams,tensorboard_log=f"runs/{run.id}")

a2c_model.learn(
    total_timesteps=config["total_timesteps"],
    callback=WandbCallback(
        gradient_save_freq=100,
        model_save_path=f"wb_logs/{run.id}", # save the model in the run directory
        verbose=2,
    ),
)
run.finish()

Using cpu device
Logging to runs/4gu9cyq5\A2C_1
------------------------------------
| time/                 |          |
|    fps                | 623      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -17      |
|    explained_variance | -0.00878 |
|    learning_rate      | 0.0005   |
|    n_updates          | 99       |
|    policy_loss        | -19.2    |
|    std                | 1        |
|    value_loss         | 98.9     |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 629      |
|    iterations         | 200      |
|    time_elapsed       | 1        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -17      |
|    explained_variance | -0.0135  |
|    learning_rate      | 0.0005   |
|    n_updates          | 1

Initial Portfolio Value : 20000 | Final Portfolio Value : 535691.0720697702 | Total rewards : 515691.0720697702 | % of profit : 2578.4553603488507
------------------------------------
| time/                 |          |
|    fps                | 618      |
|    iterations         | 1600     |
|    time_elapsed       | 12       |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       | -17      |
|    explained_variance | 0.0133   |
|    learning_rate      | 0.0005   |
|    n_updates          | 1599     |
|    policy_loss        | -30.1    |
|    std                | 1        |
|    value_loss         | 57.8     |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 618      |
|    iterations         | 1700     |
|    time_elapsed       | 13       |
|    total_timesteps    | 8500     |
| train/                |          |
|    entropy_loss       | -17.1    |
| 

Initial Portfolio Value : 20000 | Final Portfolio Value : 863244.6013850659 | Total rewards : 843244.6013850659 | % of profit : 4216.223006925329
------------------------------------
| time/                 |          |
|    fps                | 616      |
|    iterations         | 3100     |
|    time_elapsed       | 25       |
|    total_timesteps    | 15500    |
| train/                |          |
|    entropy_loss       | -17.2    |
|    explained_variance | -0.0854  |
|    learning_rate      | 0.0005   |
|    n_updates          | 3099     |
|    policy_loss        | -26.2    |
|    std                | 1.01     |
|    value_loss         | 26.1     |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 616      |
|    iterations         | 3200     |
|    time_elapsed       | 25       |
|    total_timesteps    | 16000    |
| train/                |          |
|    entropy_loss       | -17.2    |
|  

------------------------------------
| time/                 |          |
|    fps                | 628      |
|    iterations         | 4600     |
|    time_elapsed       | 36       |
|    total_timesteps    | 23000    |
| train/                |          |
|    entropy_loss       | -17.3    |
|    explained_variance | -0.00086 |
|    learning_rate      | 0.0005   |
|    n_updates          | 4599     |
|    policy_loss        | -1.13    |
|    std                | 1.02     |
|    value_loss         | 3.14e+03 |
------------------------------------
Initial Portfolio Value : 20000 | Final Portfolio Value : 439306.64155202184 | Total rewards : 419306.64155202184 | % of profit : 2096.5332077601092
------------------------------------
| time/                 |          |
|    fps                | 628      |
|    iterations         | 4700     |
|    time_elapsed       | 37       |
|    total_timesteps    | 23500    |
| train/                |          |
|    entropy_loss       | -17.3    |


------------------------------------
| time/                 |          |
|    fps                | 634      |
|    iterations         | 6100     |
|    time_elapsed       | 48       |
|    total_timesteps    | 30500    |
| train/                |          |
|    entropy_loss       | -17.3    |
|    explained_variance | 0.0013   |
|    learning_rate      | 0.0005   |
|    n_updates          | 6099     |
|    policy_loss        | 628      |
|    std                | 1.02     |
|    value_loss         | 4.26e+04 |
------------------------------------
Initial Portfolio Value : 20000 | Final Portfolio Value : 1121576.948741096 | Total rewards : 1101576.948741096 | % of profit : 5507.88474370548
------------------------------------
| time/                 |          |
|    fps                | 634      |
|    iterations         | 6200     |
|    time_elapsed       | 48       |
|    total_timesteps    | 31000    |
| train/                |          |
|    entropy_loss       | -17.3    |
|   

------------------------------------
| time/                 |          |
|    fps                | 639      |
|    iterations         | 7600     |
|    time_elapsed       | 59       |
|    total_timesteps    | 38000    |
| train/                |          |
|    entropy_loss       | -17.4    |
|    explained_variance | 0.00283  |
|    learning_rate      | 0.0005   |
|    n_updates          | 7599     |
|    policy_loss        | -303     |
|    std                | 1.03     |
|    value_loss         | 2.05e+03 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 639      |
|    iterations         | 7700     |
|    time_elapsed       | 60       |
|    total_timesteps    | 38500    |
| train/                |          |
|    entropy_loss       | -17.4    |
|    explained_variance | -0.0375  |
|    learning_rate      | 0.0005   |
|    n_updates          | 7699     |
|    policy_loss        | -133     |
|

------------------------------------
| time/                 |          |
|    fps                | 642      |
|    iterations         | 9100     |
|    time_elapsed       | 70       |
|    total_timesteps    | 45500    |
| train/                |          |
|    entropy_loss       | -17.5    |
|    explained_variance | 0.0176   |
|    learning_rate      | 0.0005   |
|    n_updates          | 9099     |
|    policy_loss        | -830     |
|    std                | 1.04     |
|    value_loss         | 1.88e+04 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 642      |
|    iterations         | 9200     |
|    time_elapsed       | 71       |
|    total_timesteps    | 46000    |
| train/                |          |
|    entropy_loss       | -17.5    |
|    explained_variance | 0.0114   |
|    learning_rate      | 0.0005   |
|    n_updates          | 9199     |
|    policy_loss        | -150     |
|

global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
time/fps,▃▄▃▂▂▂▂▂▂▁▂▁▁▂▂▃▃▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇████
train/entropy_loss,██▇█▇▇█▇▇▇▆▆▆▆▆▆▆▄▄▄▅▅▅▄▅▄▄▃▃▂▂▂▁▂▂▂▁▁▁▁
train/explained_variance,▅▅▅▆▆▆▅▇▄▅▆▅▂▅▃▂█▆▅▅▄▄▆▅▅▅▅▅▁▅▄█▅▅▄▄▅▇▅▅
train/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/policy_loss,▆▆▆▆▆▆▆▆▆▅▆▆▆▆▆▆▇▆▆▆▅▄▆▅▇▆▇█▆▅▆▆▆▁▆▆▆▆█▂
train/std,▁▁▂▁▂▂▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▄▅▅▆▆▆▇▇█▇▇▇████
train/value_loss,▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▁▁▂█▁▁▂▁▂▄
global_step,50000
time/fps,643.0
train/entropy_loss,-17.49494


In [20]:
mean_reward, std_reward = evaluate_policy(a2c_model, trade_env, n_eval_episodes=100, deterministic=True)

Initial Portfolio Value : 20000 | Final Portfolio Value : 70201.44942474365 | Total rewards : 50201.44942474365 | % of profit : 251.00724712371826
Initial Portfolio Value : 20000 | Final Portfolio Value : 70201.44942474365 | Total rewards : 50201.44942474365 | % of profit : 251.00724712371826
Initial Portfolio Value : 20000 | Final Portfolio Value : 70201.44942474365 | Total rewards : 50201.44942474365 | % of profit : 251.00724712371826
Initial Portfolio Value : 20000 | Final Portfolio Value : 70201.44942474365 | Total rewards : 50201.44942474365 | % of profit : 251.00724712371826
Initial Portfolio Value : 20000 | Final Portfolio Value : 70201.44942474365 | Total rewards : 50201.44942474365 | % of profit : 251.00724712371826
Initial Portfolio Value : 20000 | Final Portfolio Value : 70201.44942474365 | Total rewards : 50201.44942474365 | % of profit : 251.00724712371826
Initial Portfolio Value : 20000 | Final Portfolio Value : 70201.44942474365 | Total rewards : 50201.44942474365 | % of

In [21]:
# Modify Policy network

import torch.nn as nn
from wandb.integration.sb3 import WandbCallback

policy_kwargs = dict(
    net_arch=[
      dict(vf=[64, 64], pi=[64, 64]), # network architectures for actor/critic
    ],
    activation_fn=nn.Tanh,
)

config = {
    "policy_type": "MlpPolicy",
    "total_timesteps": 50000,
    "env_name": train_env,
}

hyperparams = dict(
    n_steps=5, # number of steps to collect data before updating policy
    learning_rate=0.0005,
    gamma=0.1, # discount factor
    max_grad_norm=0.5, # The maximum value for the gradient clipping
    ent_coef=0.05, # Entropy coefficient for the loss calculation
)
"""
hyperparams = dict(
    n_steps=7, # number of steps to collect data before updating policy
    learning_rate=0.001003582768121682,
    gamma=0.0018126757076625249, # discount factor
    max_grad_norm=0.2819622027161751, # The maximum value for the gradient clipping
    ent_coef=0.05, # Entropy coefficient for the loss calculation
)
"""

run = wandb.init(
    project="20240421_US_STOCK_experiment",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    monitor_gym=True,  # auto-upload the videos of agents playing the game
    save_code=True,  # optional
)

MF_P_Network_model = A2C("MlpPolicy", train_env, seed=10, verbose=1, **hyperparams,tensorboard_log=f"runs/{run.id}").learn(50000,
        callback=WandbCallback(
        gradient_save_freq=100,
        model_save_path=f"wb_logs/{run.id}",
        verbose=2,

    ))

wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


Using cpu device
Logging to runs/u95z3rv8\A2C_1
------------------------------------
| time/                 |          |
|    fps                | 652      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -17      |
|    explained_variance | -0.00629 |
|    learning_rate      | 0.0005   |
|    n_updates          | 99       |
|    policy_loss        | -23.4    |
|    std                | 1        |
|    value_loss         | 108      |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 649      |
|    iterations         | 200      |
|    time_elapsed       | 1        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -17      |
|    explained_variance | -0.0134  |
|    learning_rate      | 0.0005   |
|    n_updates          | 1

Initial Portfolio Value : 20000 | Final Portfolio Value : 553254.0998443189 | Total rewards : 533254.0998443189 | % of profit : 2666.2704992215945
------------------------------------
| time/                 |          |
|    fps                | 649      |
|    iterations         | 1600     |
|    time_elapsed       | 12       |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       | -17.1    |
|    explained_variance | 0.0176   |
|    learning_rate      | 0.0005   |
|    n_updates          | 1599     |
|    policy_loss        | -29      |
|    std                | 1        |
|    value_loss         | 57.1     |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 649      |
|    iterations         | 1700     |
|    time_elapsed       | 13       |
|    total_timesteps    | 8500     |
| train/                |          |
|    entropy_loss       | -17.1    |
| 

Initial Portfolio Value : 20000 | Final Portfolio Value : 727424.8660791995 | Total rewards : 707424.8660791995 | % of profit : 3537.1243303959973
------------------------------------
| time/                 |          |
|    fps                | 645      |
|    iterations         | 3100     |
|    time_elapsed       | 24       |
|    total_timesteps    | 15500    |
| train/                |          |
|    entropy_loss       | -17.2    |
|    explained_variance | -0.057   |
|    learning_rate      | 0.0005   |
|    n_updates          | 3099     |
|    policy_loss        | -61.7    |
|    std                | 1.01     |
|    value_loss         | 63.2     |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 646      |
|    iterations         | 3200     |
|    time_elapsed       | 24       |
|    total_timesteps    | 16000    |
| train/                |          |
|    entropy_loss       | -17.1    |
| 

------------------------------------
| time/                 |          |
|    fps                | 637      |
|    iterations         | 4600     |
|    time_elapsed       | 36       |
|    total_timesteps    | 23000    |
| train/                |          |
|    entropy_loss       | -17.2    |
|    explained_variance | 0.00199  |
|    learning_rate      | 0.0005   |
|    n_updates          | 4599     |
|    policy_loss        | 106      |
|    std                | 1.02     |
|    value_loss         | 4.73e+03 |
------------------------------------
Initial Portfolio Value : 20000 | Final Portfolio Value : 639839.5466927632 | Total rewards : 619839.5466927632 | % of profit : 3099.197733463816
------------------------------------
| time/                 |          |
|    fps                | 636      |
|    iterations         | 4700     |
|    time_elapsed       | 36       |
|    total_timesteps    | 23500    |
| train/                |          |
|    entropy_loss       | -17.2    |
|  

------------------------------------
| time/                 |          |
|    fps                | 632      |
|    iterations         | 6100     |
|    time_elapsed       | 48       |
|    total_timesteps    | 30500    |
| train/                |          |
|    entropy_loss       | -17.3    |
|    explained_variance | 0.00478  |
|    learning_rate      | 0.0005   |
|    n_updates          | 6099     |
|    policy_loss        | 398      |
|    std                | 1.02     |
|    value_loss         | 1.58e+04 |
------------------------------------
Initial Portfolio Value : 20000 | Final Portfolio Value : 750636.8909874642 | Total rewards : 730636.8909874642 | % of profit : 3653.184454937321
------------------------------------
| time/                 |          |
|    fps                | 632      |
|    iterations         | 6200     |
|    time_elapsed       | 49       |
|    total_timesteps    | 31000    |
| train/                |          |
|    entropy_loss       | -17.3    |
|  

------------------------------------
| time/                 |          |
|    fps                | 631      |
|    iterations         | 7600     |
|    time_elapsed       | 60       |
|    total_timesteps    | 38000    |
| train/                |          |
|    entropy_loss       | -17.4    |
|    explained_variance | 0.00214  |
|    learning_rate      | 0.0005   |
|    n_updates          | 7599     |
|    policy_loss        | 695      |
|    std                | 1.03     |
|    value_loss         | 1.73e+04 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 630      |
|    iterations         | 7700     |
|    time_elapsed       | 61       |
|    total_timesteps    | 38500    |
| train/                |          |
|    entropy_loss       | -17.4    |
|    explained_variance | -0.0371  |
|    learning_rate      | 0.0005   |
|    n_updates          | 7699     |
|    policy_loss        | 258      |
|

------------------------------------
| time/                 |          |
|    fps                | 630      |
|    iterations         | 9100     |
|    time_elapsed       | 72       |
|    total_timesteps    | 45500    |
| train/                |          |
|    entropy_loss       | -17.4    |
|    explained_variance | 0.0238   |
|    learning_rate      | 0.0005   |
|    n_updates          | 9099     |
|    policy_loss        | -355     |
|    std                | 1.04     |
|    value_loss         | 9.24e+03 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 630      |
|    iterations         | 9200     |
|    time_elapsed       | 73       |
|    total_timesteps    | 46000    |
| train/                |          |
|    entropy_loss       | -17.5    |
|    explained_variance | 0.00841  |
|    learning_rate      | 0.0005   |
|    n_updates          | 9199     |
|    policy_loss        | -435     |
|

In [22]:
mean_reward, std_reward = evaluate_policy(MF_P_Network_model, trade_env, n_eval_episodes=100, deterministic=True)

Initial Portfolio Value : 20000 | Final Portfolio Value : 70201.44942474365 | Total rewards : 50201.44942474365 | % of profit : 251.00724712371826
Initial Portfolio Value : 20000 | Final Portfolio Value : 70201.44942474365 | Total rewards : 50201.44942474365 | % of profit : 251.00724712371826
Initial Portfolio Value : 20000 | Final Portfolio Value : 70201.44942474365 | Total rewards : 50201.44942474365 | % of profit : 251.00724712371826
Initial Portfolio Value : 20000 | Final Portfolio Value : 70201.44942474365 | Total rewards : 50201.44942474365 | % of profit : 251.00724712371826
Initial Portfolio Value : 20000 | Final Portfolio Value : 70201.44942474365 | Total rewards : 50201.44942474365 | % of profit : 251.00724712371826
Initial Portfolio Value : 20000 | Final Portfolio Value : 70201.44942474365 | Total rewards : 50201.44942474365 | % of profit : 251.00724712371826
Initial Portfolio Value : 20000 | Final Portfolio Value : 70201.44942474365 | Total rewards : 50201.44942474365 | % of

In [23]:
# Automatic Hyperparameter Tuning(Optuna)
# In this part we will create a script that allows to search for the best hyperparameters automatically

!pip install optuna

import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

N_TRIALS = 100  # Maximum number of trials
N_JOBS = 1 # Number of jobs to run in parallel
N_STARTUP_TRIALS = 5  # Stop random sampling after N_STARTUP_TRIALS
N_EVALUATIONS = 25  # Number of evaluations during the training
N_TIMESTEPS = int(5e4)  # Training budget
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_ENVS = 3 # Number of environments to evaluate the agent
N_EVAL_EPISODES = 10 # Number of episodes to evaluate the agent
TIMEOUT = int(60 * 60*6)  # Time budget in seconds

#ENV_ID = "StockTradingEnv-v0"
ENV_ID = train_env

DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
    "seed":10,
    "env":ENV_ID,
    "tensorboard_log":"./logs",
}

from typing import Any
from typing import Dict

import gymnasium
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from stable_baselines3 import A2C
from stable_baselines3.common.callbacks import EvalCallback
from wandb.integration.sb3 import WandbCallback
from stable_baselines3.common.monitor import Monitor
import torch
import torch.nn as nn

def sample_a2c_params(trial: optuna.Trial) -> Dict[str, Any]:
    """
    Sampler for A2C hyperparameters.

    :param trial: Optuna trial object
    :return: The sampled hyperparameters for the given trial.
    """

    gamma = 1.0 - trial.suggest_float("gamma", 0.001, 0.1, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.1, 2.0, log=True)
    n_steps = trial.suggest_int("suggest_n_steps", 5, 10 , step=1, log=True)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.01, log=True)
    net_arch = trial.suggest_categorical("net_arch", ["tiny", "small"])
    activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])
    ent_coef= trial.suggest_float("ent_coef", 0.001, 0.1, log=True)

    ### YOUR CODE HERE ###
    #raise NotImplementedError("Implement sample_a2c_params()!")

    ### END OF YOUR CODE

    # Display true values
    trial.set_user_attr("gamma_", gamma)
    trial.set_user_attr("n_steps", n_steps)

    net_arch = [
        {"pi": [64], "vf": [64]}
        if net_arch == "tiny"
        else {"pi": [64, 64], "vf": [64, 64]}
    ]

    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]

    return {
        "n_steps": n_steps,
        "gamma": gamma,
        "learning_rate": learning_rate,
        "max_grad_norm": max_grad_norm,
        "ent_coef": ent_coef,
        "policy_kwargs": {
            "net_arch": net_arch,
            "activation_fn": activation_fn,


        },
    }

from stable_baselines3.common.callbacks import EvalCallback

class TrialEvalCallback(EvalCallback):
    """
    Callback used for evaluating and reporting a trial.

    :param eval_env: Evaluation environement
    :param trial: Optuna trial object
    :param n_eval_episodes: Number of evaluation episodes
    :param eval_freq:   Evaluate the agent every ``eval_freq`` call of the callback.
    :param deterministic: Whether the evaluation should
        use a stochastic or deterministic policy.
    :param verbose:
    """

    def __init__(
        self,
        eval_env: gym.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 1000,
        deterministic: bool = True,
        verbose: int = 0,
    ):

        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            # Evaluate policy (done in the parent class)
            super()._on_step()
            self.eval_idx += 1
            # Send report to Optuna
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True
    
#from wandb.keras import WandbCallback
from wandb.integration.keras import WandbCallback

def objective(trial: optuna.Trial) -> float:
    """
    Objective function using by Optuna to evaluate
    one configuration (i.e., one set of hyperparameters).

    Given a trial object, it will sample hyperparameters,
    evaluate it and report the result (mean episodic reward after training)

    :param trial: Optuna trial object
    :return: Mean episodic reward after training
    """

    kwargs = DEFAULT_HYPERPARAMS.copy()
    ### YOUR CODE HERE
    # TODO:
    # 1. Sample hyperparameters and update the default keyword arguments: `kwargs.update(other_params)`
    # 2. Create the evaluation envs
    # 3. Create the `TrialEvalCallback`

    # 1. Sample hyperparameters and update the keyword arguments
    kwargs.update(sample_a2c_params(trial))
    # Create the RL model
    Optuna_Trial_model = A2C( **kwargs)

    # 2. Create envs used for evaluation using `make_vec_env`, `ENV_ID` and `N_EVAL_ENVS`
    eval_env=train_env
    # eval_env = Monitor(gym.make(ENV_ID))
    #eval_env = Monitor(train_env)
    # 3. Create the `TrialEvalCallback` callback defined above that will periodically evaluate
    # and report the performance using `N_EVAL_EPISODES` every `EVAL_FREQ`
    # TrialEvalCallback signature:
    # TrialEvalCallback(eval_env, trial, n_eval_episodes, eval_freq, deterministic, verbose)



    eval_callback = TrialEvalCallback(
        eval_env, trial, n_eval_episodes=N_EVAL_EPISODES, eval_freq=EVAL_FREQ, deterministic=True,verbose=1,
    )

    #WandbCallback=WandbCallback(gradient_save_freq=100,model_save_path=f"wb_logs/{run.id}",verbose=2,)

    #callbacks=[WandbCallback,eval_callback]

    config = {
            "policy_type": "MlpPolicy",
            #"total_timesteps": 25000,
            "env_name": train_env,
        }


    ### END OF YOUR CODE

    nan_encountered = False
    try:
        # Train the model
        #model.learn(N_TIMESTEPS, callback=eval_callback)
        run = wandb.init(
            project="20240421_US_STOCK_experiment",
            config=config,
            sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
            monitor_gym=True,  # auto-upload the videos of agents playing the game
            save_code=True,  # optional
        )
        Optuna_Trial_model.learn(N_TIMESTEPS, callback=eval_callback)
        Optuna_Trial_model.save(f"wb_logs/{run.name}")




    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN
        print(e)
        nan_encountered = True
    finally:
        # Free memory
        Optuna_Trial_model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()


    # Update best model if the current trial has higher reward
    #global best_reward, best_model_params
    #if mean_reward > best_reward:
    #      best_reward = mean_reward
    #      best_model_params = Optuna_Trial_model.get_parameters()

    return eval_callback.last_mean_reward

In [24]:
print('Parameters Optimization Start!')

Parameters Optimization Start!


In [25]:
if run_optuma == 'yes':

    import torch as th

    # Set pytorch num threads to 1 for faster training
    th.set_num_threads(1)
    # Select the sampler, can be random, TPESampler, CMAES, ...
    sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
    # Do not prune before 1/3 of the max budget is used
    #pruner = MedianPruner(
    #    n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3
    #)

    pruner=optuna.pruners.HyperbandPruner(
                                            min_resource=1,
                                            max_resource=N_TRIALS,
                                            reduction_factor=3
        )

    # Create the study and start the hyperparameter optimization
    study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")



    try:
        study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT)
    except KeyboardInterrupt:
        pass

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print(f"  Value: {trial.value}")

    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    print("  User attrs:")
    for key, value in trial.user_attrs.items():
        print(f"    {key}: {value}")

    # Write report
    study.trials_dataframe().to_csv("study_results_a2c_optuna.csv")

    fig1 = plot_optimization_history(study)
    fig2 = plot_param_importances(study)

    fig1.show()
    fig2.show()
else:
    pass

In [26]:
print('Parameters Optimization Done!')

Parameters Optimization Done!


In [27]:
# Look into WB logs and find the latest optimized run and get the run name
import os
CWD = os.getcwd()
WB_Logs_Path = CWD + "//wb_logs//"
WB_Logs_Dir_list = os.listdir(WB_Logs_Path)
#print(WB_Logs_Dir_list)

# Remove ".zip" extension
cleaned_list = [item.replace('.zip', '') for item in WB_Logs_Dir_list]

# Sort the list based on the last 3 characters in descending order
WB_Logs_Dir_Sorted_list = sorted(cleaned_list, key=lambda x: x[-3:], reverse=True)

#print(' ')
#print(WB_Logs_Dir_Sorted_list)

import re

# Regular expression pattern for "something-something-number"
pattern = re.compile(r'^[a-z]+-[a-z]+-\d{3}$')

# Filter the list to include only items that match the pattern
WB_Logs_Dir_Filtered_list = [item for item in WB_Logs_Dir_Sorted_list if pattern.match(item)]

#print(' ')
#print(WB_Logs_Dir_Filtered_list)

# Grab the first item in the list
A2C_Model_Latest_Run = WB_Logs_Dir_Filtered_list[0]

#print(' ')
print(A2C_Model_Latest_Run)

dashing-moon-735


In [28]:
#Load Best Model
Best_A2C_Model = A2C_Model_Latest_Run
Best_model=A2C.load(f"wb_logs/{Best_A2C_Model}")

# Setup Trade Parameters

In [29]:
# Check Stocks in Tic list
print(tic)
# tic = ['AAPL','AMZN','GOLD','GOOG','GOOGL','META','MSFT','SMCI','SNOW','TSLA','TSM']

# Should be this:
# tic = ['AAPL','AMZN','GOLD','GOOG','GOOGL','META','MSFT','SMCI','SNOW','TSLA','TSM']

['AAPL', 'AMZN', 'GOLD', 'GOOG', 'GOOGL', 'META', 'MSFT', 'SMCI', 'SNOW', 'TSLA', 'TSM', 'CRM']


In [30]:
# Stock holdings
#Current_Tic_Holdings_list = [53,10,270,2,40,10,10,2,20,2,70]
print(Current_Tic_Holdings_list)

# Should look like this:
# [53,10,270,2,40,10,10,2,20,2,70]

[53, 10, 270, 2, 40, 10, 10, 2, 20, 2, 70, 10]


# Parameters 3 - Trade Parameters

In [31]:
# Environment Parameters
ticLen = len(tic)               # number of stocks in tic list
Shares_Per_Trade = 9999         # Maximum shares that can be purchased or sold in a trade
#Initial_Investment = 5959.32    # Initial Investment
Initial_Investment = 3768.32    # Initial Investment
Action_Space = ticLen           # Number of stocks

cd_cash = 12000                 # CD cash value
Algorithmic_Trading_Result = [] # Output string for algorithmic trading

# No. of Initial Investment + No. of Stocks Closing Prices + No. of Shares holding per stock + No. of stocks * No. of Technical Indicators

No_Of_Initial_Investment = 1                             # No. of Initial Investment
No_Of_Closing_prices = Action_Space                      # No. of Stocks Closing Prices
No_Of_Shares_Per_Stock = len(Current_Tic_Holdings_list)  # No. of Shares held per stock
No_Of_Stocks = Action_Space                              # No. of stocks
No_Of_Technical_Indicators = 5                           # No. of Technical Indicators

Observation_Space = No_Of_Initial_Investment + No_Of_Closing_prices + No_Of_Shares_Per_Stock + No_Of_Stocks * No_Of_Technical_Indicators
                     
# Ensure the calculation is correct
assert Observation_Space == (1 + ticLen + len(Current_Tic_Holdings_list) + ticLen * No_Of_Technical_Indicators)

duplicate_to_next_day = 'yes' # if you want to duplicate trading data to next trading day

In [32]:
#Trading Data - 2 days before current date
print(twoDaysBeforeTodays_date)
inputStartDate = twoDaysBeforeTodays_date
#inputStartDate = '2024-05-28'
dateRangeList = []
trading_data = stock_df[(stock_df.Date >= inputStartDate)]
trading_data.sort_values(["Date", "Ticker"], ignore_index=True)
trading_data.index = trading_data.Date.factorize()[0]

2024-05-31


In [33]:
!pip install pandas_market_calendars

In [34]:
if duplicate_to_next_day == 'yes':
    
    import pandas as pd
    import numpy as np
    import pandas_market_calendars as mcal

    latest_trading_date = trading_data['Date'].iloc[-1].strftime('%Y-%m-%d')

    # duplicate the latest day to the next day
    # and see whether if predicts

    latest_trading_date_df = trading_data[trading_data['Date']== latest_trading_date]
    latest_trading_date_df

    # Function to get the next trading day
    def get_next_trading_day(current_date):
        nyse = mcal.get_calendar('NYSE')
        schedule = nyse.schedule(start_date=current_date, end_date=current_date + pd.DateOffset(days=10))
        next_trading_day = schedule.iloc[1].name  # get the next trading day after the given date
        return next_trading_day

    # Duplicate the DataFrame
    new_rows = latest_trading_date_df.copy()

    # Convert Date column to datetime
    latest_trading_date_df['Date'] = pd.to_datetime(latest_trading_date_df['Date'])
    new_rows['Date'] = latest_trading_date_df['Date'].apply(get_next_trading_day)

    # Combine the original and new DataFrames
    combined_df = pd.concat([latest_trading_date_df, new_rows])

    # Reset the index
    combined_df.reset_index(drop=True, inplace=True)

    trading_data = combined_df
    trading_data.sort_values(["Date", "Ticker"], ignore_index=True)
    trading_data.index = trading_data.Date.factorize()[0]
    
else:
    pass

In [35]:
trading_data

,Date,Open,High,Low,Close,AdjClose,Volume,Ticker,volume_adi,volume_obv,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,2024-05-31,191.440002,192.570007,189.910004,192.250000,192.250000,74776921.0,AAPL,-1.279608e+13,3.644316e+10,...,2.299492,2.390772,-0.091280,-4.263600,-6.941010,2.677410,189.520314,0.501859,0.500604,6915.981339
0,2024-05-31,178.229996,179.210007,173.869995,176.440002,176.440002,58350991.0,AMZN,2.218243e+10,1.794569e+10,...,-0.392337,0.153654,-0.545991,-3.457333,-5.138151,1.680818,180.963708,-1.606070,-1.619107,6391.538140
0,2024-05-31,17.129999,17.260000,16.969999,17.090000,17.090000,15459464.0,GOLD,-4.957437e+11,1.169816e+09,...,0.484825,0.793227,-0.308402,-11.289881,-10.420045,-0.869836,17.607786,-0.116893,-0.116962,-38.894905
0,2024-05-31,173.192001,174.419998,170.970001,173.960007,173.960007,27173303.0,GOOG,7.315997e+09,9.383309e+09,...,2.030185,2.545757,-0.515571,-3.671716,-7.876583,4.204867,174.166383,0.230473,0.230208,2073.684105
0,2024-05-31,171.860001,173.059998,169.440002,172.500000,172.500000,36174235.0,GOOGL,7.752538e+09,1.148508e+10,...,2.063114,2.574123,-0.511008,-4.306067,-7.978229,3.672162,172.725041,0.226599,0.226343,2045.244527
0,2024-05-31,465.799988,469.119995,454.460114,466.829987,466.829987,16645191.0,META,-4.677736e+11,4.938520e+11,...,-0.293233,-0.460612,0.167378,-11.198798,-12.709827,1.511029,470.257292,-0.047104,-0.047116,1122.404674
0,2024-05-31,416.410004,416.630005,404.519989,415.130005,415.130005,47676297.0,MSFT,-2.362908e+12,4.909677e+09,...,0.794380,0.915615,-0.121235,10.867785,-1.274575,12.142360,424.664985,0.110930,0.110868,2665.386797
0,2024-05-31,815.770020,819.650024,760.010010,784.510010,784.510010,7168478.0,SMCI,1.488473e+08,2.845687e+08,...,-0.773555,-0.276110,-0.497445,-1.142673,-2.144296,1.001623,855.496104,-5.245548,-5.388136,12138.845998
0,2024-05-31,140.270004,142.100006,133.589996,136.179993,136.179993,14004884.0,SNOW,-5.102560e+10,1.063237e+11,...,-2.332818,-0.674064,-1.658754,23.442811,11.950284,11.492528,149.282159,-3.384182,-3.442771,-46.371048
0,2024-05-31,178.419998,180.320007,173.820099,178.080002,178.080002,67110156.0,TSLA,8.771155e+10,1.250517e+11,...,0.899642,0.955808,-0.056166,-10.414795,-9.378482,-1.036313,172.862631,-0.397109,-0.397900,11081.245274


In [36]:
# **** Stock Tracing Env 2 ****

class StockTradingEnv2(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, df, Shares_Per_Trade = 10, Initial_Investment = 10000, Action_Space = ticLen, Observation_Space = 78, day = 0,
                 Normalized_Rewards = 1e-4, verbosity = 0, mode = 'train', seed = 10, commission = 0  ):

      self.day = day
      self.df = df
      self.max_shares_per_trade = Shares_Per_Trade
      self.initial_investment = Initial_Investment
      self.Action_Space = Action_Space
      self.Observation_Space = Observation_Space
      self.normalized_rewards = Normalized_Rewards
      self.verbosity = verbosity
      self.commission = commission
      self.mode = mode
      self._seed(seed)

      #************************
      self.ticker = df.Ticker.unique()


      #Action Space
      # Action > 0  means buy shares of stock
      # Action 0 means Hold the stock
      # Action < 0 means sell shares of stock
      self.action_space = spaces.Box(low = -1, high= 1,
                                    shape=(self.Action_Space,), dtype= int)
      #Observation Space
      self.observation_space = spaces.Box(low = -np.inf, high= np.inf, shape=(self.Observation_Space,))
      #Selecting the Data for one date
      self.data = self.df.loc[self.day,:]
      #Initial Run
      self.initial = True
      #Verify if tradings days are completed or not
      self.done = False
      #Rewards
      self.reward = 0
      #Asset value after each trading day
      self.asset_memory = [self.initial_investment]
      #Rewards received for each trading day i.e profit or loss
      self.reward_memory = []
      #Saving the date for the trade
      self.date_memory = [self.data.Date.unique()[0]]
      #Initializing state of the environment
      self.state = [self.initial_investment] + self.data.AdjClose.values.tolist() + Current_Tic_Holdings_list + \
        self.data.momentum_ppo.values.tolist() + self.data.momentum_rsi.values.tolist() + \
        self.data.trend_adx.values.tolist() + self.data.trend_macd.values.tolist() + \
        self.data.trend_cci.values.tolist()
        
      self.starting_portfolio_value = self.state[0] + sum(np.array(self.state[1:self.Action_Space+1])*
                                                   np.array(self.state[self.Action_Space+1:(2*self.Action_Space)+1]))

      # Ensure the length matches the observation space
      assert len(self.state) == self.Observation_Space

      #************************
      self.keep_row_info = {}
      self.keep_rows_list = []

    def render(self, mode='human'):
      return self.state

    # This method is used to reset the values of the state to it's default after every episode
    def reset(self):
      #print(" ")
      #print("reseting self...")

      self.day = 0
      self.reward = 0
      self.data = self.df.loc[self.day,:]
      self.done = False
      self.initial = False
      self.reward_memory = []
      self.date_memory = [str(self.data.Date.unique()[0])]
      self.asset_memory = [self.initial_investment]
      #***Technical Indicators tolist (check the  observation_space num !!!)
      #***Search "self.state ="
      self.state = [self.initial_investment] + self.data.AdjClose.values.tolist() + Current_Tic_Holdings_list + \
        self.data.momentum_ppo.values.tolist() + self.data.momentum_rsi.values.tolist() + \
        self.data.trend_adx.values.tolist() + self.data.trend_macd.values.tolist() + \
        self.data.trend_cci.values.tolist()

      self.starting_portfolio_value = self.state[0] + sum(np.array(self.state[1:self.Action_Space+1])*
                                                   np.array(self.state[self.Action_Space+1:(2*self.Action_Space)+1]))
    
      #************************
      self.keep_row_info = {}
      self.keep_rows_list = []
      #print(self.state)
      return self.state

    def step(self, actions):
      #************************
      print(' ')
      print('Day: ' + str(self.day))
      dateRangeList.append(str(self.data.Date.unique()[0])[:10])
      inputEndDate = str(self.data.Date.unique()[0])[:10]
      print(str(self.data.Date.unique()[0])[:10])
      print('Available cash before trading: ' + str(self.state[0]))

      self.done = self.day >= len(self.df.Date.unique())-1
      #Use this to save the results to csv after we performed trading for all the days
      if self.done:

        keep_df = pd.DataFrame(index = [x for x in range(len(self.keep_rows_list))],data = self.keep_rows_list)

        #print('Keep DF')
        #print(keep_df)

        final_portfolio_value = self.state[0] + sum(np.array(self.state[1:self.Action_Space+1])*
                                                     np.array(self.state[self.Action_Space+1:(2*self.Action_Space)+1]))
        
        final_final_portfolio_value = final_portfolio_value + 12000
        starting_portfolio_value = self.starting_portfolio_value + 12000
        
        #total_rewards = final_portfolio_value - self.initial_investment
        total_rewards = final_final_portfolio_value - starting_portfolio_value
        profit_pct = (total_rewards*100)/starting_portfolio_value
        asset_df = pd.DataFrame(self.asset_memory)
        asset_df.columns = ['portfolio']
        asset_df['date'] = self.date_memory
        #if self.verbosity and self.mode != 'train':
        if self.verbosity :
          #print(len(self.reward_memory))
          #if self.mode == 'trade' or self.mode == 'val':

          asset_df['date'] = self.date_memory
          print(' ')
          Algorithmic_Trading_Result.append(str('Start Date : '+str(dateRangeList[0])+' | End Date : '+str(dateRangeList[-1])+' | Initial Portfolio Value : '+str(starting_portfolio_value)+' | Final Portfolio Value : '+str(final_final_portfolio_value)+' | Total rewards : '+str(total_rewards)+' | % of profit : '+str(profit_pct)))  
          print('Start Date : {} | End Date : {} | Initial Portfolio Value : {} | Final Portfolio Value : {} | Total rewards : {} | % of profit : {}'.format(dateRangeList[0], dateRangeList[-1], starting_portfolio_value, final_final_portfolio_value, total_rewards, profit_pct))
          asset_df.to_csv('{}_{}_results.csv'.format(self.mode, self.commission))

        #************************
        #keep_df = pd.DataFrame(keep_day_list, columns=['date'], index = [x for x in range(len(keep_day_list))])
        #keep_df['portfolio_before_trade'] = keep_portfolio_before_trade_list
        #keep_df['available_cash_after_trading'] = keep_available_cash_after_trading_list
        #keep_df['shares_available_per_stock_list'] = keep_shares_available_per_stock_list
        #keep_df['profit_and_loss_for_day'] = keep_profit_and_loss_for_day

        #print(keep_df)

        #keep_df.to_csv('keep_results.csv')

        return self.state, self.reward, self.done, {}

      else:

        #Calculating the portfolio value before start of trading
        #Available investment amount + sum of value of each stock held (no.of shares per stock * price of the stock on that day)
        portfolio_before_trade = self.state[0] + sum(np.array(self.state[1:self.Action_Space+1])*
                                                     np.array(self.state[self.Action_Space+1:(2*self.Action_Space)+1]))

        print('CD value: ' + str(12000))
        print('Stocks portfolio value before trading: ' + str(portfolio_before_trade))
        print('Total portfolio value before trading: ' + str(portfolio_before_trade + 12000))
        
        #print stocks and available shares
        print(' ')
        print('** Stocks and holdings before trade: **')
        for tidx in range(len(tic)):
            ticker_symbol = self.ticker[tidx]
            ticker_holdings = str(self.state[tidx+self.Action_Space+1])
            print(ticker_symbol + ': ' + ticker_holdings + ' Shares')
            
        print(' ')
        print('** Stocks Technical Indicators: **')
        for tidx in range(len(tic)):
            ticker_symbol = self.ticker[tidx]
            print(ticker_symbol)
            
            print(self.data[self.data['Ticker'] == ticker_symbol][['momentum_ppo', 'momentum_rsi','trend_adx', 'trend_macd', 'trend_cci', 'AdjClose']])
            print(' ')
            
        #************************
        keep_portfolio_before_trade = portfolio_before_trade
        #keep_portfolio_before_trade_list.append(keep_portfolio_before_trade)
        keep_available_cash_before_trading = self.state[0]

        #Extracting the indicies of sell action
        #if actions is not an array then convert it to array using np.array()
        sell_indices = np.where(actions <  0 )[0]
        #Extracting the indicies of buy action
        buy_indices = np.where(actions >  0 )[0]


        #************************
        #Hold indicies
        hold_indices = np.where(actions ==  0 )[0]



        #print(actions)
        #print(self.ticker)


        ###### Trading starts #######
        print(' ')
        print('*************** Trading start ***************')
        print(' ')
        # Initially selling the stocks to increase investment value
        
        for idx in sell_indices:

          #************************
          sell_ticker_symbol = self.ticker[idx]
          

          stock_to_sell_ticker = 'Stock_to_sell_' + str(sell_ticker_symbol)
          self.keep_row_info[stock_to_sell_ticker] = 'Yes'

          #Sell stock if price is > 0 and shares held > 0
          if self.state[idx+1] > 0 and self.state[idx+self.Action_Space+1] > 0:

            #No of shares to sell
            shares_sell = min(self.state[idx+self.Action_Space+1], abs(actions[idx]*self.max_shares_per_trade))
        
            sell_before_holdings = str(self.state[idx+self.Action_Space+1])
                  
                  
            num_of_ticker_shares_available_before_selling = 'num_of_' + str(sell_ticker_symbol) + 'shares_available_before_selling'
            self.keep_row_info[num_of_ticker_shares_available_before_selling] = str(self.state[idx+self.Action_Space+1])

            #************************
            #print('No. of shares to sell: ' + str(shares_sell))
            num_of_ticker_to_sell = 'num_of_' + str(sell_ticker_symbol) + '_to_sell:'
            self.keep_row_info[num_of_ticker_to_sell] = str(shares_sell)

            #Updating the available cash after selling the stocks
            self.state[0] += self.state[idx+1]*shares_sell*(1-self.commission)

            #Updating the available stocks after selling
            self.state[idx+self.Action_Space+1] -= shares_sell

            #************************
            #print('No. of shares available after selling: ' + str(self.state[idx+self.Action_Space+1]))
            num_of_ticker_shares_available_after_selling = 'num_of_' + str(sell_ticker_symbol) + 'shares_available_after_selling'
            self.keep_row_info[num_of_ticker_shares_available_after_selling] = str(self.state[idx+self.Action_Space+1])

            sell_after_holdings = str(self.state[idx+self.Action_Space+1])
            
            if shares_sell > 0:
                sellPrice = self.data[self.data['Ticker'] ==  sell_ticker_symbol]['AdjClose']
                print('Sold ' + str(shares_sell) + ' shares of ' + str(sell_ticker_symbol) +' at $'+str(round(sellPrice[0],2))+' a share!') 

            
            #stock_info_df = self.df
            #wanted_df_1 = stock_info_df[stock_info_df['Date'] == date_string]
            #wanted_df_2 = wanted_df_1[wanted_df_1['Ticker'] == sell_ticker_symbol]
            #wanted_df_2 = wanted_df_2[['Date', 'Ticker', 'Open', 'High', 'Low', 'AdjClose', 'Volume','momentum_ppo', 'momentum_rsi','trend_adx', 'trend_macd', 'trend_cci' ]]


            #for w_column in wanted_df_2.columns.tolist():
              #print(str(sell_ticker_symbol))
              #print(str(w_column))
              #print(str(wanted_df_2[w_column].values.tolist()[0]))
              #print(str(sell_ticker_symbol) + '_' + str(w_column)+ ' : ' + str(wanted_df_2[w_column].values.tolist()))

            #  self.keep_row_info[str(sell_ticker_symbol) + '_' + str(w_column)] = str(wanted_df_2[w_column].values.tolist()[0])
              #pass



          else:
            # print('No Shares to sell')
            pass
        #print('Buying Stock shares : ')
        for idx in buy_indices:

          #************************
          #print("Stock to buy: " + str(idx))
          buy_ticker_symbol = self.ticker[idx]
          #print('Ticker: ' + str(buy_ticker_symbol))

          stock_to_buy_ticker = 'Stock_to_buy_' + str(buy_ticker_symbol)
          self.keep_row_info[stock_to_buy_ticker] = 'Yes'

          #Buy stocks if price is > 0
          if self.state[idx+1] > 0 and self.state[0] > 0:

            #Max number of shares that can be brought with the available cash (available cash / stock price)
            max_shares_buy = self.state[0]*(1 - self.commission)//self.state[idx+1]
            #max_shares_buy = int(self.state[0]*(1 - self.commission)//self.state[idx+1]//1000)*1000

            #No of shares to buy
            shares_buy = min(max_shares_buy, actions[idx]*self.max_shares_per_trade)
            #shares_buy = ((min(max_shares_buy, actions[idx]*self.max_shares_per_trade))//1000)*1000

            buy_before_holdings = str(self.state[idx+self.Action_Space+1])
            
            num_of_ticker_shares_available_before_buying = 'num_of_' + str(buy_ticker_symbol) + 'shares_available_before_buying'
            self.keep_row_info[num_of_ticker_shares_available_before_buying] = str(self.state[idx+self.Action_Space+1])

            #************************
            #print('No. of shares to buy: ' + str(shares_buy))
            num_of_ticker_to_buy = 'num_of_' + str(buy_ticker_symbol) + '_to_buy:'
            self.keep_row_info[num_of_ticker_to_buy] = str(shares_buy)

            #Updating the available cash after buying the stocks
            self.state[0] -= self.state[idx+1]*shares_buy*(1 + self.commission)

            #Updating the available stocks after buying
            self.state[idx+self.Action_Space+1] += shares_buy

            #************************
            #print('No. of shares available after buying: ' + str(self.state[idx+self.Action_Space+1]))
            num_of_ticker_shares_available_after_buying = 'num_of_' + str(buy_ticker_symbol) + 'shares_available_after_buying'
            self.keep_row_info[num_of_ticker_shares_available_after_buying] = str(self.state[idx+self.Action_Space+1])
            
            buy_after_holdings = str(self.state[idx+self.Action_Space+1])
            
            if shares_buy > 0:
                buyPrice = self.data[self.data['Ticker'] ==  buy_ticker_symbol]['AdjClose']
                print('Bought ' + str(shares_buy) + ' shares of ' + str(buy_ticker_symbol) + ' at $' + str(round(buyPrice[0],2)) +' a share!')
            
            #stock_info_df = self.df
            #wanted_df_1 = stock_info_df[stock_info_df['Date'] == date_string]
            #wanted_df_2 = wanted_df_1[wanted_df_1['Ticker'] == buy_ticker_symbol]
            #wanted_df_2 = wanted_df_2[['Date', 'Ticker', 'Open', 'High', 'Low', 'AdjClose', 'Volume','momentum_ppo', 'momentum_rsi','trend_adx', 'trend_macd', 'trend_cci' ]]

#@@@
            #for w_column in wanted_df_2.columns.tolist():
              #print(str(sell_ticker_symbol))
              #print(str(w_column))
              #print(str(wanted_df_2[w_column].values.tolist()[0])
              #print(str(sell_ticker_symbol) + '_' + str(w_column)+ ' : ' + str(wanted_df_2[w_column].values.tolist()))

            #  self.keep_row_info[str(buy_ticker_symbol) + '_' + str(w_column)] = str(wanted_df_2[w_column].values.tolist()[0])
              #pass

          else:

            #************************
            for idx in hold_indices:
              #print("Stock to hold: " + str(idx))
              hold_ticker_symbol = self.ticker[idx]
              #print('Ticker: ' + str(hold_ticker_symbol))

              stock_to_hold_ticker = 'Stock_to_hold_' + str(hold_ticker_symbol)
              self.keep_row_info[stock_to_hold_ticker] = 'Yes'

              num_of_ticker_shares_available_for_holding = 'num_of_' + str(hold_ticker_symbol) + 'shares_available_for_holding'
              self.keep_row_info[num_of_ticker_shares_available_for_holding] = str(self.state[idx+self.Action_Space+1])

              #stock_info_df = self.df
              #wanted_df_1 = stock_info_df[stock_info_df['Date'] == date_string]
              #wanted_df_2 = wanted_df_1[wanted_df_1['Ticker'] == hold_ticker_symbol]
              #wanted_df_2 = wanted_df_2[['Date', 'Ticker', 'Open', 'High', 'Low', 'AdjClose', 'Volume','momentum_ppo', 'momentum_rsi','trend_adx', 'trend_macd', 'trend_cci' ]]

#@@@
              #for w_column in wanted_df_2.columns.tolist():
                #print(str(sell_ticker_symbol))
                #print(str(w_column))
                #print(str(wanted_df_2[w_column].values.tolist()[0]))
                #print(str(hold_ticker_symbol) + '_' + str(w_column)+ ' : ' + str(wanted_df_2[w_column].values.tolist()))

              #  self.keep_row_info[str(hold_ticker_symbol) + '_' + str(w_column)] = str(wanted_df_2[w_column].values.tolist()[0])
                #pass

            # print('No Shares purchased')
            pass
        ###### Trading ends #######
        print(' ')
        print('*************** Trading Ends ***************')
        print(' ')
        print('** Stocks and holdings after trade: **')
        for tidx in range(len(tic)):
            ticker_symbol = self.ticker[tidx]
            ticker_holdings = str(round(self.state[tidx+self.Action_Space+1]))
            print(ticker_symbol + ': ' + ticker_holdings + ' Shares')
        
        portfolio_after_trade = self.state[0] + sum(np.array(self.state[1:self.Action_Space+1])*
                                                     np.array(self.state[self.Action_Space+1:(2*self.Action_Space)+1]))
        
        print(' ')
        print('Available cash for after trading : {}'.format(self.state[0]))
        # print('Shares available per stock : '.format(np.array(self.state[self.Action_Space+1:(2*self.Action_Space)+1])))
        print('Stocks portfolio value after trading : {}'.format(portfolio_after_trade))
        print('Total portfolio value after trading: ' + str(portfolio_after_trade + 12000))
        #print('Profit or Loss for Day {} : {} is {}'.format(self.day, self.date_memory[-1], self.reward))

        #************************
        keep_available_cash_after_trading = self.state[0]
        keep_shares_available_per_stock = np.array(self.state[self.Action_Space+1:(2*self.Action_Space)+1])
        keep_profit_and_loss_for_day = self.reward
        #keep_available_cash_after_trading_list.append(keep_available_cash_after_trading)
        #keep_shares_available_per_stock_list.append(keep_shares_available_per_stock)
        #keep_profit_and_loss_for_day_list.append(Keep_profit_and_loss_for_day)




        #Setting the values for next trading date
        self.day += 1

        #************************


        self.data = self.df.loc[self.day,:]

        self.state = [self.state[0]] + self.data.AdjClose.values.tolist() + list(self.state[(self.Action_Space+1):(2*self.Action_Space)+1]) + \
                      self.data.momentum_ppo.values.tolist() + self.data.momentum_rsi.values.tolist() + \
                      self.data.trend_adx.values.tolist() + self.data.trend_macd.values.tolist() + \
                      self.data.trend_cci.values.tolist()


        portfolio_after_trade = self.state[0] + sum(np.array(self.state[1:self.Action_Space+1])*
                                                     np.array(self.state[self.Action_Space+1:(2*self.Action_Space)+1]))

        #************************
        keep_portfolio_after_trade = portfolio_after_trade
        #print('Portfolio value after trading:  ' + str(keep_portfolio_after_trade))
        #print('Available Cash before trading:  ' + str(keep_available_cash_before_trading))
        #print('Available Cash after trading:   ' + str(keep_available_cash_after_trading))
        #print('Profit and loss for day:        ' + str(keep_profit_and_loss_for_day))

        self.keep_row_info['portfolio_value_after_trading'] = str(keep_portfolio_after_trade)
        self.keep_row_info['available_cash_before_trading'] = str(keep_available_cash_before_trading)
        self.keep_row_info['available_cash_after_trading'] = str(keep_available_cash_after_trading)
        self.keep_row_info['profit_and_loss_for_day'] = str(keep_profit_and_loss_for_day)

        #Total trade in a day (profit or loss)
        self.reward = portfolio_after_trade - portfolio_before_trade
        #print('Day : {} | Reward : {}'.format(self.day-1, self.reward))
        self.reward_memory.append(self.reward)
        self.asset_memory.append(portfolio_after_trade)
        self.date_memory.append(self.data.Date.unique()[0])
        self.reward = self.reward*self.normalized_rewards     #Normalizing the reward

        self.keep_rows_list.append(self.keep_row_info)
        #print(self.keep_rows_list)

        return self.state, self.reward, self.done, {}

    def _seed(self, seed = 10):
      randomState, seed = seeding.np_random(seed)
      return [seed]

    #added for seed
    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

def render_trading(model, env, data, n_episodes = 1):
  episode_rewards = [0.0]
  obs = env.reset()
  env.render()

  for i in range(n_episodes):
    done = False
    while not done:
        action, _states = model.predict(obs)
        #print(action)
        obs, rewards, done, info = env.step(action)
        env.render()
        # Stats
        if done:
          obs = env.reset()
          #print('Episode {} Rewards {}'.format(i+1, episode_rewards[-1]))
          episode_rewards.append(0.0)
        else:
          #print(rewards)
          episode_rewards[-1] += rewards
    if (i+1)%100 == 0 and i != 0:
      print('Average reward {}'.format(np.average(episode_rewards[:i+1])))

  return episode_rewards

In [37]:
#Trading Environment
trade_env = DummyVecEnv([lambda : StockTradingEnv2(trading_data, Shares_Per_Trade,
                                                  Initial_Investment, Action_Space,
                                                  Observation_Space, verbosity=1, mode='trade',
                                                  Normalized_Rewards= Normalized_Rewards,
                                                  commission = commission)])

trade_env = VecNormalize(trade_env, norm_reward=False)

In [38]:
#VS Buy and Hold function
def vs_buy_and_hold(start_date, end_date, holdings_list, starting_cash, other_cash):
    
    from datetime import datetime, timedelta
    
    #### Find start total portfolio ####
    starting_stock_dataframe = pd.DataFrame()
    original_start_date_object = datetime.strptime(start_date, '%Y-%m-%d')
    new_start_date_object = original_start_date_object + timedelta(days=1)
    day_after_start_date = new_start_date_object.strftime('%Y-%m-%d')

    for stock_ticker1 in tic:
        temp_data = yfinance.download(stock_ticker1, start=trading_start_date, end=day_after_start_date, progress=False)
        temp_data['Ticker'] = stock_ticker1
        starting_stock_dataframe = starting_stock_dataframe.append(temp_data)
    
    starting_stock_dataframe['Shares'] = holdings_list
    starting_subtotal_portfolio = sum(starting_stock_dataframe['Shares'] * starting_stock_dataframe['Adj Close'])
    ##print(starting_stock_dataframe[['Adj Close','Shares']])
    starting_total_portfolio = starting_subtotal_portfolio + starting_cash + other_cash
    
    #### Find end total portfolio ####
    ending_stock_dataframe = pd.DataFrame()
    original_end_date_object = datetime.strptime(end_date, '%Y-%m-%d')
    new_end_date_object = original_end_date_object + timedelta(days=1)
    day_after_end_date = new_end_date_object.strftime('%Y-%m-%d')

    for stock_ticker2 in tic:
        temp_data2 = yfinance.download(stock_ticker2, start=trading_end_date, end=day_after_end_date, progress=False)
        temp_data2['Ticker'] = stock_ticker2
        ending_stock_dataframe = ending_stock_dataframe.append(temp_data2)
        
    ending_stock_dataframe['Shares'] = holdings_list
    ending_subtotal_portfolio = sum(ending_stock_dataframe['Shares'] * ending_stock_dataframe['Adj Close'])
    ##print(ending_stock_dataframe[['Adj Close','Shares']])
    ending_total_portfolio = ending_subtotal_portfolio + starting_cash + other_cash
    
    totalRewards = ending_total_portfolio - starting_total_portfolio
    pctOfProfit = (totalRewards*100)/starting_total_portfolio
    
    return ('Start Date : {} | End Date : {} | Initial Portfolio Value : {} | Final Portfolio Value : {} | Total rewards : {} | % of profit : {}'.format(start_date, end_date, starting_total_portfolio, ending_total_portfolio, totalRewards, pctOfProfit))

In [42]:
# Algorithmic Trading result
mean_reward, std_reward = evaluate_policy(Best_model, trade_env, n_eval_episodes=1, deterministic=True)

 
Day: 0
2024-05-31
Available cash before trading: 3768.32
CD value: 12000
Stocks portfolio value before trading: 53969.76942474365
Total portfolio value before trading: 65969.76942474366
 
** Stocks and holdings before trade: **
AAPL: 53 Shares
AMZN: 10 Shares
GOLD: 270 Shares
GOOG: 2 Shares
GOOGL: 40 Shares
META: 10 Shares
MSFT: 10 Shares
SMCI: 2 Shares
SNOW: 20 Shares
TSLA: 2 Shares
TSM: 70 Shares
CRM: 10 Shares
 
** Stocks Technical Indicators: **
AAPL
   momentum_ppo  momentum_rsi  trend_adx  trend_macd  trend_cci  AdjClose
0      2.299492     67.294615  33.423601    4.254256  79.732172    192.25
 
AMZN
   momentum_ppo  momentum_rsi  trend_adx  trend_macd   trend_cci    AdjClose
0     -0.392337     36.745153  10.671988   -0.715091 -199.716751  176.440002
 
GOLD
   momentum_ppo  momentum_rsi  trend_adx  trend_macd  trend_cci  AdjClose
0      0.484825     49.885153  16.046388    0.082935 -11.645975     17.09
 
GOOG
   momentum_ppo  momentum_rsi  trend_adx  trend_macd  trend_cci    A

In [40]:
print(Algorithmic_Trading_Result[0])

Start Date : 2024-05-31 | End Date : 2024-06-03 | Initial Portfolio Value : 65969.76942474366 | Final Portfolio Value : 65969.76942474366 | Total rewards : 0.0 | % of profit : 0.0


In [41]:
if duplicate_to_next_day == 'yes':
    pass
else:
    trading_start_date = dateRangeList[0]
    trading_end_date = dateRangeList[-1]
    print(vs_buy_and_hold(trading_start_date, trading_end_date, Current_Tic_Holdings_list, Initial_Investment, cd_cash))